In [4]:
from copy import deepcopy
from random import choice
import time
import random
import string

def make_encoding_table():
    table = []
    for i in range(26):
        row = []
        c = 0
        for j in range(26):
            if j - i < 0:
                row.append(26-i+c)
                c+=1
            else:
                row.append(j-i)
        table.append(row)
    return table


In [5]:
 
class Decoder :
    number_of_generation = 0
    def __init__(self, global_text, encoded_text_lines, key_length):
        self.encoded_text_lines = deepcopy(encoded_text_lines)
        self.global_text = deepcopy(global_text)
        self.key_length = key_length
        self.population_size = 50
        self.mutation_rate = 0.7
        self.solved = False
        self.answer = []
        self.dictionary = set()
        self.best_fitnesses = []
        self.encoding_table = make_encoding_table()
        self.count_words()
        self.create_dict()
    
    def count_words(self):
        count =0
        for line in self.encoded_text_lines:
            for word in line.split():
                count += 1
        self.text_words_count = count

    def create_dict(self):
        bad_chars = [';', ",",':', '!', "*" ,"^", '-', "_", "?", "{" , "}" , "#" , ">" , "+" , "@" , "`", "&" , "|" , "$" , "]"  , "=" , "(" , ")" , '"' , "." , "[" , "/" ,'0','1','2','3','4','5','6','7','8','9']

        bad_words = [' your ', ' yors ', ' yorself ', ' yorselves ', ' he ', ' him ', ' his ', ' himself ', ' she ', 
        " she's ", ' her ', ' hers ', ' herself ', ' it ', " it's ", ' its ', ' itself ', ' they ', ' them ', 
        ' their ', ' theirs ', ' themselves ', ' what ', ' which ', ' who ', ' whom ', ' this ', 
        ' that ', " that'll ", ' these ', ' those ', ' am ', ' is ', ' are ', ' was ', ' were ', ' be ',
        ' been ', ' being ', ' have ', ' has ', ' had ', ' having ', ' do ', ' does ', ' did ', ' doing ', ' an ', ' the ', ' and ', ' but ', ' if ', ' or ', ' because ', ' as ', ' until '," a " ,
        ' while ', ' of ', ' at ',' in ',' on ', " to "]
        # using replace() to
        # remove bad_chars
        for i in bad_chars :
            self.global_text = self.global_text.replace(i, ' ')
        # for i in bad_words :
        #     self.global_text = self.global_text.replace(i, ' ')
        for word in self.global_text.split():
            self.dictionary.add(word)
        #print(self.dictionary)

    def making_first_generation(self):
        self.chromosomes = []
        for i in range(self.population_size):
            chromosome = ''.join(random.choices(string.ascii_lowercase, k = self.key_length))     
            self.chromosomes.append(chromosome)
    
    def fitness(self,original_text):
        score = 0
        for line in original_text :
            for word in line :
                if word in self.dictionary:
                    score +=1 
        return score
    
    def get_original_text(self,key) : 
        original_text = []
        original_letter = ''
        original_word = ""
        original_line = []
        count = 0

        for line in self.encoded_text_lines:
            for word in line.split():
                for letter in word:
                    if letter not in [',','.','?',':','-','_','(',')','[',']','0','1','2','3','4','5','6','7','8','9','—']:
                        
                        if letter.isupper():
                            col = ord(letter) - 65
                            row = ord(key[count%self.key_length]) - 97
                            
                            original_letter = self.encoding_table[row][col]
                            original_letter = chr(original_letter + 65)
                            original_letter = original_letter
                        else:
                            col = ord(letter.lower()) - 97
                            row = ord(key[count%self.key_length].lower()) - 97
                            
                            original_letter = self.encoding_table[row][col]
                            original_letter = chr(original_letter + 97)
                        count += 1

                    else:
                        original_letter = letter
                    original_word += original_letter
                    original_letter = ''
                original_line.append(original_word)
                original_word = ""
            original_text.append(original_line)
            original_line = []
        return original_text

    def crossover(self, key1, key2):
        crossover_child = key1
        crossover_child2 = key2
        first_index = random.randint(0,13)
        second_index = random.randint(0,13)
        third_index = random.randint(0,13)
        indexes = []
        while second_index == first_index:
            second_index = random.randint(0,13)
        while third_index == first_index or third_index == second_index:
            third_index = random.randint(0,13)
        indexes.append(first_index)
        indexes.append(second_index)
        indexes.append(third_index)
        indexes.sort()
        a = list(key1)
        b = list(key2)
        c = list(crossover_child)
        a[0:indexes[0]] = b[0:indexes[0]]
        a[indexes[1]:indexes[2]] = b[indexes[1]:indexes[2]]

        b[0:indexes[0]] = c[0:indexes[0]]
        b[indexes[1]:indexes[2]] = c[indexes[1]:indexes[2]]

        crossover_child = "".join(a)
        crossover_child2 = "".join(b)
        return crossover_child,crossover_child2

    def mutation(self, key):
        b = list(key)
        first_index = random.randint(0,13)
        b[first_index]= (random.choice(string.ascii_letters)).lower()

        key = "".join(b)
        return key

    def finish(self, key, original_text):
        self.solved = True
        self.final_key = key
        self.original_text = original_text

    def get_best_fitnesses(self):
        return self.best_fitnesses

    def get_solved(self):
        return self.solved  
        
    def making_new_generation(self):
        Decoder.number_of_generation += 1
        ranked_base_population = []

        for key in self.chromosomes:
            original_text = self.get_original_text(key)
            ranked_base_population.append([self.fitness(original_text), key, original_text])

        ranked_base_population.sort(reverse=1)
        total_rank_sum = sum(range(1, len(ranked_base_population) + 1))

        self.best_fitnesses.append(ranked_base_population[0])
        if ranked_base_population[0][0] >= 0.8*self.text_words_count:
            #print(ranked_base_population[0][2])
            print("key : ", ranked_base_population[0][1])
            print(self.get_original_text(ranked_base_population[0][1]))
            self.finish(ranked_base_population[0][1],ranked_base_population[0][2])
        print(
            f"generation number {Decoder.number_of_generation} : best fitness : {ranked_base_population[0][0]}")

        key_probabilities = []
        for i in range(len(ranked_base_population)):
            key_probabilities.append((len(ranked_base_population)- i )/total_rank_sum)
        selected = random.choices(ranked_base_population,weights=key_probabilities,k=self.population_size)
        new_generation = []

        for i in range(self.population_size//2):
            if i<5 :
                new_generation.append(ranked_base_population[i][1])
                continue
            map1, map2 = random.sample(selected, 2)
            key1 = map1[1]
            key2 = map2[1]
            child_crossover,child_crossover2 = self.crossover(key1, key2)
            mutation_result = self.mutation(child_crossover)
            mutation_result2 = self.mutation(child_crossover2)

            new_generation.append(deepcopy(mutation_result))
            new_generation.append(deepcopy(mutation_result2))

        self.chromosomes = deepcopy(new_generation)
         
    def decode(self):
        self.making_first_generation()
        while self.get_solved() != True:
            self.making_new_generation()
        return self.best_fitnesses[2]
        



In [6]:
# def decode(encoded_text_lines):
#     decoder = Decode(encoded_text_lines)
#     decoder.making_first_generation()
#     a = time.time()
#     while decoder.get_solved() != True:
#         decoder.making_new_generation()
#     b = time.time()
#     print(f"time : {b-a}")

# decode(encoded_text_lines)
encoded_text=open('encoded_text.txt').readlines()
global_text=open('global_text.txt').read()
d = Decoder(global_text,encoded_text,14)
decodedText=d.decode()


generation number 1 : best fitness : 19
generation number 2 : best fitness : 24
generation number 3 : best fitness : 24
generation number 4 : best fitness : 34
generation number 5 : best fitness : 34
generation number 6 : best fitness : 34
generation number 7 : best fitness : 34
generation number 8 : best fitness : 34
generation number 9 : best fitness : 37
generation number 10 : best fitness : 37
generation number 11 : best fitness : 49
generation number 12 : best fitness : 61
generation number 13 : best fitness : 61
generation number 14 : best fitness : 64
generation number 15 : best fitness : 64
generation number 16 : best fitness : 85
generation number 17 : best fitness : 85
generation number 18 : best fitness : 85
generation number 19 : best fitness : 85
generation number 20 : best fitness : 151
generation number 21 : best fitness : 151
generation number 22 : best fitness : 151
generation number 23 : best fitness : 157
generation number 24 : best fitness : 157
generation number 25